In [5]:
%pip install transformers
%pip install seaborn
%pip install "datasets==1.6.2"

distutils: /opt/conda/include/python3.6m/UNKNOWN
sysconfig: /opt/conda/include/python3.6m
user = False
home = None
root = None
prefix = None
distutils: /opt/conda/include/python3.6m/UNKNOWN
sysconfig: /opt/conda/include/python3.6m
user = False
home = None
root = None
prefix = None
Note: you may need to restart the kernel to use updated packages.
distutils: /opt/conda/include/python3.6m/UNKNOWN
sysconfig: /opt/conda/include/python3.6m
user = False
home = None
root = None
prefix = None
distutils: /opt/conda/include/python3.6m/UNKNOWN
sysconfig: /opt/conda/include/python3.6m
user = False
home = None
root = None
prefix = None
Note: you may need to restart the kernel to use updated packages.
distutils: /opt/conda/include/python3.6m/UNKNOWN
sysconfig: /opt/conda/include/python3.6m
user = False
home = None
root = None
prefix = None
distutils: /opt/conda/include/python3.6m/UNKNOWN
sysconfig: /opt/conda/include/python3.6m
user = False
home = None
root = None
prefix = None
Note: you may need to 

In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
import torch
import os
import transformers
import numpy as np
import seaborn as sns
import pandas as pd
from transformers import file_utils
from types import SimpleNamespace
from datasets import Dataset
import utils

In [8]:
MODEL_FILE_NAME = 'pytorch_model.bin'
model_dir = '..'
batch_size = 64

In [9]:
model = utils.get_default_model()

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
state_file = os.path.join(model_dir, MODEL_FILE_NAME)
state_dict = torch.load(state_file, map_location=device)
model.load_state_dict(state_dict)
model = model.to(device)

In [11]:
tokenizer = model.tokenizer
pt_tensor_type = transformers.file_utils.TensorType.PYTORCH

In [12]:
df = pd.read_csv('../test.csv')
df = utils.preprocess_df(df)

In [ ]:
prob_list = []
model.eval()
with torch.no_grad():
    for i in range(0, len(df), batch_size):
        print(f'batch #{i+1}')
        batch_dict = {col: list(df.loc[i:i+batch_size, col]) for col in df.columns}
        batch_input = utils.tokenize_fn(batch_dict, model.tokenizer, return_tensors=pt_tensor_type)
        outputs = model(**batch_input)
        probabilities = torch.nn.functional.softmax(outputs.logits, dim=1)
        prob_list.append(probabilities)

batch #1
[2021-09-08 20:04:21.731 pytorch-1-6-cpu-py36--ml-t3-medium-370ee60fbc7a856e8f67ac271515:432 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-09-08 20:04:22.066 pytorch-1-6-cpu-py36--ml-t3-medium-370ee60fbc7a856e8f67ac271515:432 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.


In [1]:
prob_list

NameError: name 'prob_list' is not defined

In [1]:
prob_disaster = np.vstack(prob_list)[:,1]

NameError: name 'np' is not defined

In [2]:
sns.histplot(prob_disaster) 

NameError: name 'sns' is not defined

In [146]:
np.sum(prob_disaster>0.5)/len(prob_disaster)

0.35243640821330063

In [150]:
df['target'] = [int(x) for x in prob_disaster>0.5]

In [151]:
df_submission = df.loc[:,['id','target']]

In [153]:
df_submission.to_csv('submission.csv',index=False)

### Predict top 50% 